In [1]:
import polars as pl
from typing import Literal, Sequence
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [2]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [3]:
num_cars = 10
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head())
print(data.collect().limit(5))

naive plan: (run LazyFrame.explain(optimized=True) to see the optimized plan)

SLICE[offset: 0, len: 5]
  DF ["car_id", "timestamp", "latitude", "longitude"]; PROJECT */5 COLUMNS; SELECTION: "None"
shape: (5, 5)
┌────────┬────────────────────────────┬───────────┬────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---        ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64        ┆ str           │
╞════════╪════════════════════════════╪═══════════╪════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00        ┆ 30.620572 ┆ -88.35519  ┆ null          │
│ car_0  ┆ 2023-01-01T00:01:22.758642 ┆ 30.620572 ┆ -88.35519  ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:01:05.074075 ┆ 30.620572 ┆ -88.35519  ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:09:07.834960 ┆ 30.620572 ┆ -88.35519  ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:30:00        ┆ 30.615397 ┆ -88.357

## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [4]:
#data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime))
#print(data.collect())

data = data.with_columns(
    pl.when(pl.col("timestamp").str.contains(r"\."))
    .then(pl.col("timestamp"))
    .otherwise(pl.col("timestamp") + ".000000")
    .alias("timestamp")
)

print(data.collect())

data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f"))
print(data.collect())



shape: (60_789, 5)
┌────────┬────────────────────────────┬───────────┬─────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude   ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---         ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64         ┆ str           │
╞════════╪════════════════════════════╪═══════════╪═════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00.000000 ┆ 30.620572 ┆ -88.35519   ┆ null          │
│ car_0  ┆ 2023-01-01T00:01:22.758642 ┆ 30.620572 ┆ -88.35519   ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:01:05.074075 ┆ 30.620572 ┆ -88.35519   ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:09:07.834960 ┆ 30.620572 ┆ -88.35519   ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:30:00.000000 ┆ 30.615397 ┆ -88.357069  ┆ engine_check  │
│ …      ┆ …                          ┆ …         ┆ …           ┆ …             │
│ car_9  ┆ 2023-01-07T23:38:51.379114 ┆ 17.631816 ┆ -112.46266  ┆ null         

/tmp/ipykernel_1342/3203420204.py:13: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  data = data.with_columns(pl.col("timestamp").str.strptime(pl.Datetime, "%Y-%m-%dT%H:%M:%S.%f"))


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [5]:
#A function to order data by id_car and timestamp
def order_data(data):
    data = data.sort("car_id", "timestamp")
    return data
#Ordena las filas por car_id y timestamp
data = order_data(data)
df_pre_time = data.group_by("car_id", maintain_order=True).agg(pl.all(), pl.col("timestamp").shift(1).alias("previous_timestamp"))
#print(agrupacion.collect().limit(5))
#Crea la tiempo previo y agrupa por car_id
columnas_a_explotar = [col for col in df_pre_time.columns if col != 'car_id']
df_pre_time = df_pre_time.explode(columnas_a_explotar)
print("\nDespués de explode:")
print(df_pre_time.collect().limit(10))
#Resta el timestamp actual con el timestamp previo, creando la columna notification_time
df_pre_time = df_pre_time.with_columns((pl.col("timestamp") - pl.col("previous_timestamp")).alias("notification_time"))
print("\nDespués de restar:")

#Copio el Df para trabajar con el mas adelante
data = df_pre_time.clone()
print(data.collect())


Después de explode:
shape: (10, 6)
┌────────┬────────────────────────┬───────────┬────────────┬───────────────┬───────────────────────┐
│ car_id ┆ timestamp              ┆ latitude  ┆ longitude  ┆ notification  ┆ previous_timestamp    │
│ ---    ┆ ---                    ┆ ---       ┆ ---        ┆ ---           ┆ ---                   │
│ str    ┆ datetime[ns]           ┆ f64       ┆ f64        ┆ str           ┆ datetime[ns]          │
╞════════╪════════════════════════╪═══════════╪════════════╪═══════════════╪═══════════════════════╡
│ car_0  ┆ 2023-01-01 00:00:00    ┆ 30.620572 ┆ -88.35519  ┆ null          ┆ null                  │
│ car_0  ┆ 2023-01-01             ┆ 30.620572 ┆ -88.35519  ┆ tire_pressure ┆ 2023-01-01 00:00:00   │
│        ┆ 00:01:05.000074075     ┆           ┆            ┆               ┆                       │
│ car_0  ┆ 2023-01-01             ┆ 30.620572 ┆ -88.35519  ┆ low_fuel      ┆ 2023-01-01            │
│        ┆ 00:01:22.000758642     ┆           ┆        

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [6]:
#Agrego las columnas de latitud y longitud previas
df_distance = data.group_by("car_id", maintain_order=True).agg(pl.all(), pl.col("latitude").shift(1).alias("prevlatitude"), pl.col("longitude").shift(1).alias("prevlongitude"))
columnas_a_explotar = [col for col in df_distance.columns if col != 'car_id']
df_distance = df_distance.explode(columnas_a_explotar)
print("\nDespués de explode:")
print(df_distance.collect())


Después de explode:


shape: (60_789, 9)
┌────────┬────────────┬───────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ timestamp  ┆ latitude  ┆ longitude  ┆ … ┆ previous_ ┆ notificat ┆ prevlatit ┆ prevlongi │
│ ---    ┆ ---        ┆ ---       ┆ ---        ┆   ┆ timestamp ┆ ion_time  ┆ ude       ┆ tude      │
│ str    ┆ datetime[n ┆ f64       ┆ f64        ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│        ┆ s]         ┆           ┆            ┆   ┆ datetime[ ┆ duration[ ┆ f64       ┆ f64       │
│        ┆            ┆           ┆            ┆   ┆ ns]       ┆ ns]       ┆           ┆           │
╞════════╪════════════╪═══════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ 2023-01-01 ┆ 30.620572 ┆ -88.35519  ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│        ┆ 00:00:00   ┆           ┆            ┆   ┆           ┆           ┆           ┆           │
│ car_0  ┆ 2023-01-01 ┆ 30.620572 ┆ -88.35519  ┆ … ┆ 2023-01-0 ┆ 1m 5s  

In [7]:
import polars as pl
from math import radians, sin, cos, sqrt, atan2

# Función Haversine para calcular la distancia en km
def haversine(lat1, lon1, lat2, lon2):
    if None in (lat1, lon1, lat2, lon2):
        return None

    R = 6371.0  # Radio de la Tierra en km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Calcula la distancia solo si todas las coordenadas están presentes
distance_expr = pl.when(
    pl.col("latitude").is_not_null() &
    pl.col("longitude").is_not_null() &
    pl.col("prevlatitude").is_not_null() &
    pl.col("prevlongitude").is_not_null()
).then(
    pl.struct(["latitude", "longitude", "prevlatitude", "prevlongitude"]).apply(
        lambda row: haversine(row["latitude"], row["longitude"], row["prevlatitude"], row["prevlongitude"])
    )
).otherwise(pl.lit(None)).alias("distance_km")

# Aplicar la expresión al DataFrame
df_with_distance = df_distance.with_columns(distance_expr)

print("\nDespués de calcular la distancia:")
print(df_with_distance.collect().limit(10))



Después de calcular la distancia:


/tmp/ipykernel_1342/1424608340.py:24: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.struct(["latitude", "longitude", "prevlatitude", "prevlongitude"]).apply(
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


shape: (10, 10)
┌────────┬────────────┬───────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ timestamp  ┆ latitude  ┆ longitude  ┆ … ┆ notificat ┆ prevlatit ┆ prevlongi ┆ distance_ │
│ ---    ┆ ---        ┆ ---       ┆ ---        ┆   ┆ ion_time  ┆ ude       ┆ tude      ┆ km        │
│ str    ┆ datetime[n ┆ f64       ┆ f64        ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│        ┆ s]         ┆           ┆            ┆   ┆ duration[ ┆ f64       ┆ f64       ┆ f64       │
│        ┆            ┆           ┆            ┆   ┆ ns]       ┆           ┆           ┆           │
╞════════╪════════════╪═══════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ 2023-01-01 ┆ 30.620572 ┆ -88.35519  ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│        ┆ 00:00:00   ┆           ┆            ┆   ┆           ┆           ┆           ┆           │
│ car_0  ┆ 2023-01-01 ┆ 30.620572 ┆ -88.35519  ┆ … ┆ 1m 5s     ┆ 30.620572 

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [12]:
#Definir el intervalo de tiempo. En este caso 10 mins 
interval = "10m"

# Agrupar por 'car_id' y por intervalos de tiempo en 'timestamp'
result_notification = df_with_distance.groupby_dynamic(
    #Agrupamos por car_id
    by="car_id",
    #Esta opción indica la columna que se utilizará para determinar los límites de los intervalos de tiempo
    index_column="timestamp",
    #Definimos el intervalo de tiempo
    every=interval,
    period=interval,

    closed="both",
    #Esto asegura que los límites exactos del intervalo, 
    #como el comienzo de la hora o el día exacto, estén incluidos en el agrupamiento
    include_boundaries=True
).agg([
    pl.col("notification_time").mean().alias("mean_time_ns"),
    pl.col("notification_time").median().alias("median_time_ns"),
    pl.col("notification_time").var().alias("variance_time_ns"),
    pl.col("notification_time").max().alias("max_time_ns"),
    pl.col("notification_time").min().alias("min_time_ns")
])

result_distance = df_with_distance.groupby_dynamic(
    # Agrupamos por car_id
    by="car_id",
    # Esta opción indica la columna que se utilizará para determinar los límites de los intervalos de tiempo
    index_column="timestamp",
    # Definimos el intervalo de tiempo
    every=interval,
    period=interval,
    closed="both",
    # Esto asegura que los límites exactos del intervalo, 
    # como el comienzo de la hora o el día exacto, estén incluidos en el agrupamiento
    include_boundaries=True
).agg([
    pl.col("distance_km").mean().alias("mean_distance_km"),
    pl.col("distance_km").median().alias("median_distance_km"),
    pl.col("distance_km").var().alias("variance_distance_km"),
    pl.col("distance_km").max().alias("max_distance_km"),
    pl.col("distance_km").min().alias("min_distance_km")
])


print(result_notification.collect())
print(result_distance.collect())

/tmp/ipykernel_1342/2780507269.py:5: DeprecationWarning: `groupby_dynamic` is deprecated. It has been renamed to `group_by_dynamic`.
  result_notification = df_with_distance.groupby_dynamic(
/tmp/ipykernel_1342/2780507269.py:5: DeprecationWarning: `truncate` is deprecated and will be removed in a future version. Please replace `truncate=True` with `label='left'` to silence this warning.
  result_notification = df_with_distance.groupby_dynamic(
/tmp/ipykernel_1342/2780507269.py:26: DeprecationWarning: `groupby_dynamic` is deprecated. It has been renamed to `group_by_dynamic`.
  result_distance = df_with_distance.groupby_dynamic(
/tmp/ipykernel_1342/2780507269.py:26: DeprecationWarning: `truncate` is deprecated and will be removed in a future version. Please replace `truncate=True` with `label='left'` to silence this warning.
  result_distance = df_with_distance.groupby_dynamic(
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpr

shape: (8_301, 9)
┌────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ _lower_bou ┆ _upper_bou ┆ timestamp ┆ … ┆ median_ti ┆ variance_ ┆ max_time_ ┆ min_time_ │
│ ---    ┆ ndary      ┆ ndary      ┆ ---       ┆   ┆ me_ns     ┆ time_ns   ┆ ns        ┆ ns        │
│ str    ┆ ---        ┆ ---        ┆ datetime[ ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│        ┆ datetime[n ┆ datetime[n ┆ ns]       ┆   ┆ duration[ ┆ duration[ ┆ duration[ ┆ duration[ │
│        ┆ s]         ┆ s]         ┆           ┆   ┆ ns]       ┆ ns]       ┆ ns]       ┆ ns]       │
╞════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ 2022-12-31 ┆ 2023-01-01 ┆ 2022-12-3 ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│        ┆ 23:50:00   ┆ 00:00:00   ┆ 1         ┆   ┆           ┆           ┆           ┆           │
│        ┆            ┆            ┆ 23:50:00  ┆   ┆           ┆         